In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

In [2]:
units_dict = {
    "Group ID":"Group Number in Subfind Catalogs", 
    "Group Mass":"Physical mass from Group_M_TopHat200 -- 1e10 Msun",
    "Group Radius":"Physical radius from Group_R_TopHat200 -- kpc",
    "Group Nsubs":"Number of subhalos in group",
    "Sub1 ID":  "Subhalo ID at current snapshot",
    "Sub2 ID":  "Subhalo ID at current snapshot",
    "Sub1 Mass": "Subhalo mass at current snapshot -- 1e10 Msun",
    "Sub2 Mass": "Subhalo mass at current snapshot -- 1e10 Msun",
    "Sub1 Stellar Mass": "Stellar masses from abundance matching -- 1e10 Msun",
    "Sub2 Stellar Mass": "Stellar masses from abundance matching -- 1e10 Msun",
    "Sub1 Pos": "Subhalo physical position in box x,y,z -- kpc",
    "Sub2 Pos": "Subhalo physical position in box x,y,z -- kpc",
    "Sub1 Vel": "Subhalo velocity in vx, vy, vz -- km/s",
    "Sub2 Vel": "Subhalo velocity in vx, vy, vz -- km/s",
    "Sub1 MassType": "Mass of bound particles - gas, DM, empty, tracers, stars, BHs -- in 1e10 Msun",
    "Sub2 MassType": "Mass of bound particles - gas, DM, empty, tracers, stars, BHs -- in 1e10 Msun",
    "Separation": "Physical separation between primary and secondary in kpc",
    "Comoving Separation":"Comoving separation between primary and secondary in ckpc",
    "RelVel": "Relative velocity between primary and secondary in km/s",
    "Stellar Mass Ratio": "Stellar mass ratio of secondary over primary", # in this case- it's by primary subhalo mass
    "Realization": "Stellar mass realization (0-1000)",
    "Sub1 BHMass": "Sum of the masses of all blackholes -- 1e10 Msun",
    "Sub2 BHMass": "Sum of the masses of all blackholes -- 1e10 Msun",
    "Sub1 BHMdot": "Instantaneous accretion rates of all blackholes -- 1e10 Msun / 0.978Gyr",
    "Sub2 BHMdot": "Instantaneous accretion rates of all blackholes -- 1e10 Msun / 0.978Gyr",
    "Sub1 SFR": "Sum of the individual SFRs of all gas cells in subhalo -- Msun / yr",
    "Sub2 SFR": "Sum of the individual SFRs of all gas cells in subhalo -- Msun / yr",
    "Sub1 SFRinRad": "Sum of SFRs of all gas cells within twice the stellar half mass radius -- Msun / yr",
    "Sub2 SFRinRad": "Sum of SFRs of all gas cells within twice the stellar half mass radius -- Msun / yr",
    "Sub1 GasMetallicity": "Mz/Mtot, where Z = any element above He within 2x stellar half mass radius -- unitless",
    "Sub2 GasMetallicity": "Mz/Mtot, where Z = any element above He within 2x stellar half mass radius -- unitless",
    "TripleFlag": "0 if no tertiary with mass ratio > 1:10 of secondary, 1 if large tertiary, 2 if other problem occurred"
    }

# z=1.5 sample (snapshot 40)

In [4]:
snapshot = 40
snapdata = h5py.File(f"../../pears/data/pairs/TNG_{snapshot}_10.hdf5","r")


pairs = {}
for key, val in snapdata['pairs']['hydro'].items():
    pairs[key]=np.array(val)
    
redshift = snapdata["Header"].attrs["Redshift"]
    
snapdata.close()
    
maskk  = pairs['Realization']==-1    

medianpairs = {}
for key in pairs.keys():
    medianpairs[key] = pairs[key][maskk]
    
# make functions to get data at the requested snapshot
def get_primmask(primstells, size):
    if size == "dwarf":
        mask = (primstells > 0.01) & (primstells < 0.5)
    elif size == "massive":
        mask = (primstells > 0.5) & (primstells < 10)
    return mask

def get_groupmask(groupmass, size):
    if size == "dwarf":
        mask = (groupmass > 8) & (groupmass < 50)
    elif size == "massive":
        mask = (groupmass > 100) & (groupmass < 650)
    return mask

# get out the things that are in dwarf groups:
group_mask = get_groupmask(medianpairs['Group Mass'],"dwarf")
primary_mask = get_primmask(medianpairs['Sub1 Stellar Mass'], "dwarf")
major_mask = (medianpairs['Sub2 Stellar Mass']/medianpairs['Sub1 Stellar Mass'] > 1/4)

fullmask = group_mask&primary_mask&major_mask

print(len(pairs['Sub1 Stellar Mass']))
print(len(medianpairs['Sub1 Stellar Mass']))
print(len(medianpairs['Sub1 Stellar Mass'][group_mask]))
print(len(medianpairs['Sub1 Stellar Mass'][group_mask&primary_mask]))
print(len(medianpairs['Sub1 Stellar Mass'][group_mask&primary_mask&major_mask]))
print(len(medianpairs['Sub1 Stellar Mass'][fullmask]))

# make data file
f = h5py.File(f"../data/lowmass_{snapshot}.hdf5", 'w')
header_dict = {"Details":f"Low mass major pairs at z={redshift:.1f}",
    "Snapshot":snapshot,
        "Redshift":redshift,
        "Simulation":"TNG100-1"}

dset = f.create_group('/Header')
for key in header_dict.keys():
    dset.attrs[key] = header_dict[key]
    
for key, val in medianpairs.items():
    val = np.array(val)[fullmask]
    dset = f.create_dataset(f'/{key}', 
                            shape=val.shape,
                            dtype=val.dtype)
    dset.attrs[key] = units_dict[key]
    dset[:] = val
    
f.close()


219468
18289
14819
14733
1330
1330


# z=1 sample (snapshot 50)

In [5]:
snapshot = 50
snapdata = h5py.File(f"../../pears/data/pairs/TNG_{snapshot}_10.hdf5","r")


pairs = {}
for key, val in snapdata['pairs']['hydro'].items():
    pairs[key]=np.array(val)
    
redshift = snapdata["Header"].attrs["Redshift"]
    
snapdata.close()
    
maskk  = pairs['Realization']==-1    

medianpairs = {}
for key in pairs.keys():
    medianpairs[key] = pairs[key][maskk]
    
# make functions to get data at the requested snapshot
def get_primmask(primstells, size):
    if size == "dwarf":
        mask = (primstells > 0.01) & (primstells < 0.5)
    elif size == "massive":
        mask = (primstells > 0.5) & (primstells < 10)
    return mask

def get_groupmask(groupmass, size):
    if size == "dwarf":
        mask = (groupmass > 8) & (groupmass < 50)
    elif size == "massive":
        mask = (groupmass > 100) & (groupmass < 650)
    return mask

# get out the things that are in dwarf groups:
group_mask = get_groupmask(medianpairs['Group Mass'],"dwarf")
primary_mask = get_primmask(medianpairs['Sub1 Stellar Mass'], "dwarf")
major_mask = (medianpairs['Sub2 Stellar Mass']/medianpairs['Sub1 Stellar Mass'] > 1/4)

fullmask = group_mask&primary_mask&major_mask

print(len(pairs['Sub1 Stellar Mass']))
print(len(medianpairs['Sub1 Stellar Mass']))
print(len(medianpairs['Sub1 Stellar Mass'][group_mask]))
print(len(medianpairs['Sub1 Stellar Mass'][group_mask&primary_mask]))
print(len(medianpairs['Sub1 Stellar Mass'][group_mask&primary_mask&major_mask]))
print(len(medianpairs['Sub1 Stellar Mass'][fullmask]))

# make data file
f = h5py.File(f"../data/lowmass_{snapshot}.hdf5", 'w')
header_dict = {"Details":f"Low mass major pairs at z={redshift:.1f}",
    "Snapshot":snapshot,
        "Redshift":redshift,
        "Simulation":"TNG100-1"}

dset = f.create_group('/Header')
for key in header_dict.keys():
    dset.attrs[key] = header_dict[key]
    
for key, val in medianpairs.items():
    val = np.array(val)[fullmask]
    dset = f.create_dataset(f'/{key}', 
                            shape=val.shape,
                            dtype=val.dtype)
    dset.attrs[key] = units_dict[key]
    dset[:] = val
    
f.close()


219816
18318
14444
14263
1149
1149


# z = 2 (snapshot 33) 

In [6]:
snapshot = 33
snapdata = h5py.File(f"../../pears/data/pairs/TNG_{snapshot}_10.hdf5","r")


pairs = {}
for key, val in snapdata['pairs']['hydro'].items():
    pairs[key]=np.array(val)
    
redshift = snapdata["Header"].attrs["Redshift"]
    
snapdata.close()
    
maskk  = pairs['Realization']==-1    

medianpairs = {}
for key in pairs.keys():
    medianpairs[key] = pairs[key][maskk]
    
# make functions to get data at the requested snapshot
def get_primmask(primstells, size):
    if size == "dwarf":
        mask = (primstells > 0.01) & (primstells < 0.5)
    elif size == "massive":
        mask = (primstells > 0.5) & (primstells < 10)
    return mask

def get_groupmask(groupmass, size):
    if size == "dwarf":
        mask = (groupmass > 8) & (groupmass < 50)
    elif size == "massive":
        mask = (groupmass > 100) & (groupmass < 650)
    return mask

# get out the things that are in dwarf groups:
group_mask = get_groupmask(medianpairs['Group Mass'],"dwarf")
primary_mask = get_primmask(medianpairs['Sub1 Stellar Mass'], "dwarf")
major_mask = (medianpairs['Sub2 Stellar Mass']/medianpairs['Sub1 Stellar Mass'] > 1/4)

fullmask = group_mask&primary_mask&major_mask

print(len(pairs['Sub1 Stellar Mass']))
print(len(medianpairs['Sub1 Stellar Mass']))
print(len(medianpairs['Sub1 Stellar Mass'][group_mask]))
print(len(medianpairs['Sub1 Stellar Mass'][group_mask&primary_mask]))
print(len(medianpairs['Sub1 Stellar Mass'][group_mask&primary_mask&major_mask]))
print(len(medianpairs['Sub1 Stellar Mass'][fullmask]))

# make data file
f = h5py.File(f"../data/lowmass_{snapshot}.hdf5", 'w')
header_dict = {"Details":f"Low mass major pairs at z={redshift:.1f}",
    "Snapshot":snapshot,
        "Redshift":redshift,
        "Simulation":"TNG100-1"}

dset = f.create_group('/Header')
for key in header_dict.keys():
    dset.attrs[key] = header_dict[key]
    
for key, val in medianpairs.items():
    val = np.array(val)[fullmask]
    dset = f.create_dataset(f'/{key}', 
                            shape=val.shape,
                            dtype=val.dtype)
    dset.attrs[key] = units_dict[key]
    dset[:] = val
    
f.close()


201780
16815
14052
13933
1419
1419


In [7]:
f.close()